### Import Libraries

In [ ]:
import datatable as dt  # pip install datatable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
from lightgbm import LGBMClassifier

In [ ]:
%%time
train_pd = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv")

In [ ]:
%%time
test_pd = pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv")

In [ ]:
memory_usage_train = train_pd.memory_usage(deep=True) / 1024 ** 2
memory_usage_test = test_pd.memory_usage(deep=True) / 1024 ** 2

memory_usage_train.head(7)

In [ ]:
memory_usage_train.sum()

### Reduce the memory of train and test data which is loaded using pandas

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
reduced_train_pd = reduce_memory_usage(train_pd, verbose=True)
reduced_test_pd = reduce_memory_usage(test_pd, verbose=True)

In [ ]:
y=reduced_train_pd['target'].reset_index(drop=True)
X=reduced_train_pd.drop(['id','target'],axis=1).reset_index(drop=True)

In [ ]:
test=reduced_test_pd.drop('id',axis=1)

In [ ]:
from sklearn.model_selection import StratifiedKFold

auc=[]

params={'reg_alpha': 8.158768860412389, 'reg_lambda': 8.793022151019823, 
        'colsample_bytree': 0.2, 'subsample': 0.4, 'learning_rate': 0.05,
   'max_depth': -1, 'num_leaves': 24, 'min_child_samples': 68, 'cat_smooth': 91,
        'objective': 'binary', 'random_state': 48,'n_estimators': 20000,'n_jobs': -1}
preds=np.zeros(test.shape[0])
skf = StratifiedKFold(n_splits=10)
n=0
for train_index, test_index in skf.split(X, y):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = LGBMClassifier(**params)
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=200,eval_metric="auc",verbose=50)
    print(model.predict_proba(X_test))
    auc.append(roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))
    preds += model.predict_proba(test)[:,1]
    print(skf.n_splits)
    print(f"fold: {n+1}, auc: {auc[n]}")
    n+=1

In [ ]:
preds += preds/kf.n_splits

In [ ]:
submission=pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')
submission['target'] = preds
submission.to_csv("submission.csv", index=False)
submission